In [194]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from os.path import exists as DoesPathExist
train_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv"
train_scores_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv"
test_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv"
sample_submission_path = "/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv"

if DoesPathExist(train_logs_path):
    train_logs = pd.read_csv(train_logs_path)
else:
    train_logs = pd.read_csv("./data/train_logs.csv")

if DoesPathExist(train_scores_path):
    train_scores = pd.read_csv(train_scores_path)
else:
    train_scores = pd.read_csv("./data/train_scores.csv")

if DoesPathExist(test_logs_path):
    test_logs = pd.read_csv(test_logs_path)
else:
    test_logs = pd.read_csv("./data/test_logs.csv")
    
if DoesPathExist(sample_submission_path):
    sample_submission = pd.read_csv(sample_submission_path)
else:
    sample_submission = pd.read_csv("./data/sample_submission.csv")
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)

In [195]:
train_logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [196]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [197]:
train_logs.shape

cols_to_drop = ["down_event", "up_event"]
for col in cols_to_drop:
    train_logs = train_logs.drop(col, axis=1) 
train_logs.shape

(8405898, 9)

## Data Analysis

In [198]:
train_groups: pd.DataFrame.groupby = train_logs.groupby(["id"])
input_counts = train_groups.activity.count()
print(input_counts)

id
001519c8    2557
0022f953    2454
0042269b    4136
0059420b    1556
0075873a    2531
            ... 
ffb8c745    4739
ffbef7e5    2604
ffccd6fd    3063
ffec5b38    3242
fff05981    3619
Name: activity, Length: 2471, dtype: int64


In [217]:
def FeatureEngineering(df: pd.DataFrame):
    """Computes the feature engineering for training.

    Args:
        df (pd.DataFrame): Input dataframe.

    Returns:
        pd.DataFrame: This will be prepped with everything needed to be inputed as x-values for a dataframe.
    """
    groups = df.groupby("id")
    tails = groups.tail(1)
    insertCount = groups.apply(lambda x: (x["activity"] == "Input").sum())
    result = pd.DataFrame(
        {
            "id": df["id"].unique(),
            "num_events": tails["event_id"],
            "word_count": tails["word_count"],
           # "Remove/Cut" : removal_count
        }
    )

    result = pd.merge(result, pd.DataFrame(insertCount), on="id")


    return result

train_engineered = FeatureEngineering(df=train_logs)
train_engineered.head()

id
001519c8    2010
0022f953    1938
0042269b    3515
0059420b    1304
0075873a    1942
            ... 
ffb8c745    3588
ffbef7e5    2395
ffccd6fd    2849
ffec5b38    2895
fff05981    2452
Length: 2471, dtype: int64


,id,num_events,word_count,0
0,001519c8,2557,255,2010
1,0022f953,2454,320,1938
2,0042269b,4136,404,3515
3,0059420b,1556,206,1304
4,0075873a,2531,252,1942


In [200]:
def NormalizeTrainData(input: pd.DataFrame) -> pd.DataFrame:
    min_max_scaler = preprocessing.MinMaxScaler()
    input["num_events"] = min_max_scaler.fit_transform(input[["num_events"]])
    input["word_count"] = min_max_scaler.fit_transform(input[["word_count"]])
    return input

train_data = NormalizeTrainData(train_engineered)

In [201]:
def OneHotEncoding(n: float):
    result = [0]*12
    result[int(n / 0.5) - 1] = 1
    return result

x_train = train_data.sort_values("id").drop("id", axis=1)
raw_scores = train_scores.sort_values("id").drop("id", axis=1)
y_train = np.array([OneHotEncoding(float(y)) for y in raw_scores['score']])

In [202]:
num_train = len(y_train)
x_valid = x_train[:int(num_train*0.2)]
y_valid = y_train[:int(num_train*0.2)]
x_train = x_train[int(num_train*0.2):]
y_train = y_train[int(num_train*0.2):]

In [203]:
model = Sequential([
    Dense(units=512, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=12, activation="sigmoid")
])

In [204]:
predictions = model(np.array(x_train)).numpy()
predictions
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
loss_fn(y_train, predictions).numpy()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, verbose=2)

Epoch 1/50
62/62 - 0s - loss: 2.0396 - accuracy: 0.2398 - 275ms/epoch - 4ms/step
Epoch 2/50
62/62 - 0s - loss: 1.8219 - accuracy: 0.2883 - 105ms/epoch - 2ms/step
Epoch 3/50
62/62 - 0s - loss: 1.8147 - accuracy: 0.2822 - 122ms/epoch - 2ms/step
Epoch 4/50
62/62 - 0s - loss: 1.8105 - accuracy: 0.2797 - 125ms/epoch - 2ms/step
Epoch 5/50
62/62 - 0s - loss: 1.7738 - accuracy: 0.3045 - 122ms/epoch - 2ms/step
Epoch 6/50
62/62 - 0s - loss: 1.7822 - accuracy: 0.3015 - 123ms/epoch - 2ms/step
Epoch 7/50
62/62 - 0s - loss: 1.7958 - accuracy: 0.2863 - 132ms/epoch - 2ms/step
Epoch 8/50
62/62 - 0s - loss: 1.7662 - accuracy: 0.3025 - 137ms/epoch - 2ms/step
Epoch 9/50
62/62 - 0s - loss: 1.7899 - accuracy: 0.3055 - 166ms/epoch - 3ms/step
Epoch 10/50
62/62 - 0s - loss: 1.7751 - accuracy: 0.3065 - 182ms/epoch - 3ms/step
Epoch 11/50
62/62 - 0s - loss: 1.8001 - accuracy: 0.2903 - 163ms/epoch - 3ms/step
Epoch 12/50
62/62 - 0s - loss: 1.7701 - accuracy: 0.3060 - 159ms/epoch - 3ms/step
Epoch 13/50
62/62 - 0s - 

In [205]:
results = model.evaluate(x_valid, y_valid)
for a in range(len(results)):
    print(f"{model.metrics_names[a]}: {results[a]:.3f}")

predictions: np.array = model.predict(x_valid)
scaled_Pred = []
for p in predictions:
    max = p.max()
    scaled_Pred.append(np.array([1 if x == max else 0 for x in p]))


16/16 [==============================] - 0s 909us/step - loss: 1.7357 - accuracy: 0.3239
loss: 1.736
accuracy: 0.324
16/16 [==============================] - 0s 805us/step


In [206]:
# Generate the submission
test_data = FeatureEngineering(test_logs)
test_data = NormalizeTrainData(test_data)
predictions = []
for index, row in test_data.iterrows():
    x_data = [x for x in row[1:]]
    pred = model.predict([x_data])
    max = pred[0].max()
    pred = [1 if x == max else 0 for x in pred[0]]
    result = 0.5
    for index, p in enumerate(pred):
        if p == 1:
            result = 0.5 * index + 0.5
    
    predictions.append((row[0], result)) # (Id, result)
print(predictions)

1/1 [==============================] - 0s 19ms/step
[('0000aaaa', 1.0), ('2222bbbb', 5.5), ('4444cccc', 5.5)]


In [207]:
ids = [p[0] for p in predictions]
scores = [p[1] for p in predictions]
# ids = [1, 2, 3]
# scores = [0.5, 3.0, 4.5]
sample_submission["id"] = ids
sample_submission["score"] = scores
sample_submission[["id", "score"]].to_csv("submission.csv", index=False)